In [1]:
import os
import sys
import subprocess
import idb
import pefile
import lief
import pandas as pd
import numpy as np

pe_path = "C:\\Users\\Christopher\\ChromeSetup.exe"
ida_path = os.path.join(os.environ['PROGRAMFILES'], "IDA 7.2", "idat.exe")

pe = pefile.PE(pe_path, fast_load=True)
binary = lief.parse(pe_path)
subprocess.call([ida_path, "-B", pe_path])

0

In [2]:
#Parse PE Section

sections = []
print("Section Name \t SizeOfRawData")

for section in pe.sections:
    sections.append([section.Name.decode('UTF-8'), section.SizeOfRawData])
    print(section.Name.decode('UTF-8'), "\t\t", section.SizeOfRawData)

Section Name 	 SizeOfRawData
.text    		 83456
.rdata   		 28160
.data    		 2560
.rsrc    		 1199616
.reloc   		 4608


In [3]:
#Parse Import Address Table

imports = []
pe.parse_data_directories()

for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(entry.dll.decode('UTF-8'))
    for imp in entry.imports:
        imports.append([entry.dll.decode('UTF-8'), hex(imp.address), imp.name.decode('UTF-8')])
        print('\t', hex(imp.address), imp.name.decode('UTF-8'))

KERNEL32.dll
	 0x416000 QueryPerformanceCounter
	 0x416004 GetCurrentProcessId
	 0x416008 GetCurrentThreadId
	 0x41600c GetSystemTimeAsFileTime
	 0x416010 InitializeSListHead
	 0x416014 IsDebuggerPresent
	 0x416018 UnhandledExceptionFilter
	 0x41601c SetUnhandledExceptionFilter
	 0x416020 GetStartupInfoW
	 0x416024 IsProcessorFeaturePresent
	 0x416028 GetModuleHandleW
	 0x41602c GetCurrentProcess
	 0x416030 TerminateProcess
	 0x416034 RtlUnwind
	 0x416038 GetLastError
	 0x41603c SetLastError
	 0x416040 EnterCriticalSection
	 0x416044 LeaveCriticalSection
	 0x416048 DeleteCriticalSection
	 0x41604c InitializeCriticalSectionAndSpinCount
	 0x416050 TlsAlloc
	 0x416054 TlsGetValue
	 0x416058 TlsSetValue
	 0x41605c TlsFree
	 0x416060 FreeLibrary
	 0x416064 GetProcAddress
	 0x416068 LoadLibraryExW
	 0x41606c RaiseException
	 0x416070 GetStdHandle
	 0x416074 WriteFile
	 0x416078 GetModuleFileNameW
	 0x41607c GetModuleFileNameA
	 0x416080 MultiByteToWideChar
	 0x416084 WideCharToMultiByte
	 0x

AttributeError: 'NoneType' object has no attribute 'decode'

In [ ]:
# Parse Dynamic Import using IDA Python Script

In [4]:
# Run the script

subprocess.call([ida_path, "-B", "-Sscript.py", pe_path])

0

In [5]:
# Retrieving the output

read = open('output.txt', encoding="utf8")
dynamic_imports = read.read().split(',')

dynamic_imports

['[[4199393L',
 " 'call    esi ; VerSetConditionMask']",
 ' [4199397L',
 " 'call    esi ; VerSetConditionMask']",
 ' [4199401L',
 " 'call    esi ; VerSetConditionMask']",
 ' [4203173L',
 " 'call    eax']",
 ' [4205336L',
 " 'call    edx']",
 ' [4205545L',
 " 'call    edi ; GetLastError']",
 ' [4205585L',
 " 'call    edi ; GetLastError']",
 ' [4206821L',
 " 'call    eax']",
 ' [4207156L',
 " 'call    esi ; CloseHandle']",
 ' [4207161L',
 " 'call    esi ; CloseHandle']",
 ' [4216009L',
 " 'call    edi']",
 ' [4216053L',
 " 'call    edi']",
 ' [4217234L',
 " 'call    esi ; ___DestructExceptionObject']",
 ' [4242103L',
 " 'call    esi']]"]

In [6]:
print(sections)

[['.text\x00\x00\x00', 28672], ['.rdata\x00\x00', 24576], ['.data\x00\x00\x00', 8192], ['.rsrc\x00\x00\x00', 163840]]


In [7]:
print(imports)

[['KERNEL32.dll', '0x408018', 'GetFileSizeEx'], ['KERNEL32.dll', '0x40801c', 'CreateFileA'], ['KERNEL32.dll', '0x408020', 'InitializeCriticalSection'], ['KERNEL32.dll', '0x408024', 'DeleteCriticalSection'], ['KERNEL32.dll', '0x408028', 'GetFileSize'], ['KERNEL32.dll', '0x40802c', 'LeaveCriticalSection'], ['KERNEL32.dll', '0x408030', 'EnterCriticalSection'], ['KERNEL32.dll', '0x408034', 'SizeofResource'], ['KERNEL32.dll', '0x408038', 'LockResource'], ['KERNEL32.dll', '0x40803c', 'LoadResource'], ['KERNEL32.dll', '0x408040', 'FindResourceA'], ['KERNEL32.dll', '0x408044', 'SetErrorMode'], ['KERNEL32.dll', '0x408048', 'VirtualAlloc'], ['KERNEL32.dll', '0x40804c', 'VirtualFree'], ['KERNEL32.dll', '0x408050', 'FreeLibrary'], ['KERNEL32.dll', '0x408054', 'HeapAlloc'], ['KERNEL32.dll', '0x408058', 'GetProcessHeap'], ['KERNEL32.dll', '0x40805c', 'GetModuleHandleA'], ['KERNEL32.dll', '0x408060', 'SetLastError'], ['KERNEL32.dll', '0x408064', 'VirtualProtect'], ['KERNEL32.dll', '0x408068', 'IsBadR

In [6]:
# Write output to text file

file = open('analysis_output.log','w')

file.write('[section] = [')
for i in sections:
    file.write(str(i))
    
file.write(']')

file.write('[API] = [')
for i in imports:
    file.write(str(i))

file.write(']')

file.write('[dynamic_import] = [')
for i in dynamic_imports:
    file.write(str(i))

file.write(']')

file.close()